In [1]:
import pandas as pd
from dataAug.tools import dataAugForTestSMOTE
from processTools import processRealData
import torch
from ModelClassify_5 import ModelClassify
from targeTools import testThresholdFive, Accuracy
from dataAug.tools import dataAugForTestRandomOverSampler, dataAugForTestADASYN

In [2]:
feature_pd = pd.read_csv('/home/kongge/projects/new_protT5/data/AAC_578.csv')
labels_pd = pd.read_csv("/home/kongge/projects/new_protT5/data/mutil_label_578.csv")

In [3]:
feature_pd.shape

(578, 20)

In [4]:
smote_multiple = {}
for item in range(1, 5, 2):
    G_feature, G_label = dataAugForTestSMOTE(feature_pd, labels_pd, item, 20)
    R_feature, R_label = processRealData(feature_pd, labels_pd, feature_num=20)

    train_feature = pd.concat([R_feature, G_feature], axis=0)
    train_label = pd.concat([R_label, G_label], axis=0)

    test_df = pd.read_csv("/home/kongge/projects/new_protT5/data/AAC_test_mutil_label_122_finally.csv")

    from torch.utils.data import TensorDataset, DataLoader
    datasetTrain = TensorDataset(torch.tensor(train_feature.values), torch.tensor(train_label.values))
    batch_size = len(train_feature)
    shuffle = True
    dataloaderTrain = DataLoader(datasetTrain, batch_size=batch_size, shuffle=True)

    datasetTest = TensorDataset(torch.tensor(test_df.iloc[:,:20].values), torch.tensor(test_df.iloc[:,-4:].values))
    # 创建 DataLoader 对象，设置批量大小和是否打乱数据
    batch_size = len(datasetTest)
    dataloaderTest = DataLoader(datasetTest, batch_size=batch_size, shuffle=False)

    model = ModelClassify(feature_num=20)
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.0001)


    import warnings
    warnings.filterwarnings("ignore")
    GMList = {}
    for epoch in range(200):
        model.train()
        total_loss = 0.0
        for idx, data in enumerate(dataloaderTrain, 0):
            inputs, labels = data
            labels = labels.float()
            inputs = inputs.float()
            out = model(inputs)
            loss = criterion(out, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloaderTrain)
        threshold = 0.5
        labels_cov = torch.where(out > threshold, torch.tensor(1), torch.tensor(0))
        print(f"Epoch [{epoch+1}/{200}], Average Loss: {avg_loss:.4f}, ACC: {Accuracy(labels.int(), labels_cov)}")
        GMScore1 = testThresholdFive(epoch, model, dataloaderTest)
        GMList[epoch] = GMScore1

    best_key = max(GMList, key=GMList.get)
    best_value = GMList[best_key]
    smote_multiple[item] = best_value

/home/kongge/.conda/envs/kongge_pytorch_cpu/lib/python3.7/site-packages/imblearn/utils/_validation.py:314: UserWarning: After over-sampling, the number of samples (398) in class 0 will be larger than the number of samples in the majority class (class #1 -> 379)
  f"After over-sampling, the number of samples ({n_samples})"
/home/kongge/.conda/envs/kongge_pytorch_cpu/lib/python3.7/site-packages/imblearn/utils/_validation.py:314: UserWarning: After over-sampling, the number of samples (466) in class 0 will be larger than the number of samples in the majority class (class #1 -> 345)
  f"After over-sampling, the number of samples ({n_samples})"


Epoch [1/200], Average Loss: 0.7168, ACC: 0.2631399782135075
epoch:0,bestThreshold:[0.45, 0.44, 0.48, 0.52], GM:0.4499453604221344, OAA:0.02459016393442623, ACC:0.394808743169399, F1:0.5393443703651428
Epoch [2/200], Average Loss: 0.7136, ACC: 0.26776960784313714
epoch:1,bestThreshold:[0.05, 0.05, 0.46, 0.05], GM:0.43054643273353577, OAA:0.0, ACC:0.3422131147540985, F1:0.5010929107666016
Epoch [3/200], Average Loss: 0.7180, ACC: 0.26252723311546844
epoch:2,bestThreshold:[0.05, 0.46, 0.41, 0.53], GM:0.40491804480552673, OAA:0.03278688524590164, ACC:0.31147540983606564, F1:0.4508199393749237
Epoch [4/200], Average Loss: 0.7199, ACC: 0.26402505446623065
epoch:3,bestThreshold:[0.05, 0.51, 0.4, 0.5], GM:0.3642076849937439, OAA:0.0, ACC:0.2903005464480873, F1:0.4262295663356781
Epoch [5/200], Average Loss: 0.7196, ACC: 0.2665441176470588
epoch:4,bestThreshold:[0.05, 0.05, 0.44, 0.52], GM:0.4425683617591858, OAA:0.0, ACC:0.3504098360655739, F1:0.512022078037262
Epoch [6/200], Average Loss: 0.

In [5]:
smote_multiple

{1: [tensor(0.4831),
  0.19672131147540983,
  0.4631147540983607,
  tensor(0.4713),
  tensor(0.7295),
  tensor(0.5546)],
 3: [tensor(0.4631),
  0.05737704918032787,
  0.39549180327868877,
  tensor(0.3996),
  tensor(0.9221),
  tensor(0.5410)]}

In [1]:
Gm_list = {
    1: [10,50],
    2: [100,10],
    3: [50, 40]
}

In [4]:
best_key = max(Gm_list, key=Gm_list.get)
best_value = Gm_list[best_key]

In [5]:
best_value

[100, 10]

In [7]:
import torch
x = torch.tensor(0.55)

In [8]:
x

tensor(0.5500)

In [9]:
x.item()

0.550000011920929

In [30]:
import random
import numpy as np
from dataAug.tools import selectSameLabel


def test_aug(feature_label, multiple):
    feature = feature_label.iloc[:, :1424]
    label = feature_label.iloc[:, -4:]

    multi_label_samples = label[(label.sum(axis=1) >= 2)]
    multi_label_indices = multi_label_samples.index
    multi_features_samples = feature.loc[multi_label_indices]

    true_feature = feature.iloc[:15, :]
    true_label = label.iloc[:15, :]
    true_feature = true_feature.drop(12)
    true_label = true_label.drop(12)

    multi_features_samples = pd.concat([multi_features_samples, true_feature], axis=0)
    multi_label_samples = pd.concat([multi_label_samples, true_label], axis=0)

    feature = multi_features_samples
    label = multi_label_samples
    label_sames = selectSameLabel(label)

    new_X = np.zeros((len(feature) * multiple, feature.shape[1]))
    new_y = np.zeros((len(feature) * multiple, label.shape[1]))
    i = 0

    # 筛选多标签
    for labels, samples in label_sames.items():
        for x in samples:
            true_feature = feature.loc[x]
            for _ in range(multiple):
                ratio = random.uniform(0, 0.5)
                new_X[i] = np.array(true_feature + true_feature * ratio)
                new_y[i] = np.array(labels)
                i = i + 1
    new_y = new_y.astype(int)
    new_X = pd.DataFrame(new_X)
    new_X_column_names = [str(i) for i in range(feature.shape[1])]
    new_X.columns = new_X_column_names
    new_y = pd.DataFrame(new_y)
    new_y.columns = ['envelope', 'lumen', 'stroma', 'thylakoid_membrane']
    return new_X, new_y

In [31]:
import pandas as pd
test_df = pd.read_csv("/home/kongge/projects/new_protT5/data/DPC_T5_test_mutil_label_122_finally.csv")
new_testX, new_testY = test_aug(test_df, 3)
print(new_testX.shape)
print(new_testY.shape)

(69, 1424)
(69, 4)
